In [1]:
# Chave da API no Azure
subscription_key = 'ab2d5fffc2a564285aba72cbe0a11e2f6'

In [2]:
import urllib.parse, http.client, json

cabecalho = {
    'Ocp-Apim-Subscription-Key':subscription_key,
    'Content-Type':'application/json'
}

# Cria uma faceId para uma face qualquer (Bill gates)

In [26]:
body = {
    'url':'https://www.biography.com/.image/t_share/MTIwNjA4NjMzODIyNjc2NDky/bill-gates-9307520-1-402.jpg' # Bill Gates
}

params = urllib.parse.urlencode({
    'returnFaceId':True,
    'returnFaceLandmarks':True,
    'returnFaceAttributes':'age,gender,makeup,smile,glasses,accessories'
})

url = 'idfaceapi.cognitiveservices.azure.com'

endpoint = '/face/v1.0/detect?%s'

try:
    conn = http.client.HTTPSConnection(url)
    conn.request('POST', endpoint % params, str(body), cabecalho)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    bill_gates_face_id = data[0]['faceId'] 
    print(bill_gates_face_id) # Apresenta o faceId da imagem do Bill Gates.
    print(json.dumps(data, sort_keys=True, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

da0b71e5-f740-4d2b-b9e4-f4f544d1a7ac
[
  {
    "faceAttributes": {
      "accessories": [
        {
          "confidence": 0.98,
          "type": "glasses"
        }
      ],
      "age": 57.0,
      "gender": "male",
      "glasses": "ReadingGlasses",
      "makeup": {
        "eyeMakeup": false,
        "lipMakeup": false
      },
      "smile": 1.0
    },
    "faceId": "da0b71e5-f740-4d2b-b9e4-f4f544d1a7ac",
    "faceLandmarks": {
      "eyeLeftBottom": {
        "x": 430.1,
        "y": 593.2
      },
      "eyeLeftInner": {
        "x": 491.4,
        "y": 581.4
      },
      "eyeLeftOuter": {
        "x": 370.6,
        "y": 580.6
      },
      "eyeLeftTop": {
        "x": 430.1,
        "y": 551.6
      },
      "eyeRightBottom": {
        "x": 766.3,
        "y": 584.9
      },
      "eyeRightInner": {
        "x": 699.6,
        "y": 575.5
      },
      "eyeRightOuter": {
        "x": 824.6,
        "y": 561.7
      },
      "eyeRightTop": {
        "x": 761.3,
        "y

# Cria grupo

In [5]:
grupo_id = 'grupo_pessoas'

grupo_corpo = {
    'name':'Grupo de pessoas'
}

try:
    conn = http.client.HTTPSConnection('idfaceapi.cognitiveservices.azure.com')
    conn.request('PUT', '/face/v1.0/persongroups/%s' % grupo_id, str(grupo_corpo), cabecalho)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

b''


# Adicionando pessoas ao grupo

In [20]:
# Cria pessoa.
pessoa_corpo = {
    'name':'Elizabeth'
}

pessoa_endpoint = '/face/v1.0/persongroups/{0}/persons'.format(grupo_id)

try:
    conn = http.client.HTTPSConnection('idfaceapi.cognitiveservices.azure.com')
    conn.request('POST', pessoa_endpoint, str(pessoa_corpo), cabecalho)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

{'personId': 'ef263804-9976-456e-9554-2e1c39d2bde2'}


Processei o código acima 3 vezes, de modo a criar as pessoas Angela, Bill e Elizabeth.

In [23]:
# Lista pessoas do grupo.
try:
    conn = http.client.HTTPSConnection('idfaceapi.cognitiveservices.azure.com')
    conn.request('GET', pessoa_endpoint, "", cabecalho)
    response = conn.getresponse()
    jsonData = response.read()
    pessoas_dados = json.loads(jsonData)
    print(pessoas_dados)
    conn.close()
except Exception as ex:
    print(ex)

[{'personId': 'ab9243d6-97bb-40a2-9ad7-7f9166fc1c71', 'persistedFaceIds': ['962828cd-c7aa-4cb3-b151-c17b7a04464d', 'e80923d6-8865-4676-84c0-d953a0ad0fc3', 'f456604b-a4be-4eb8-9680-6398a2aafb97'], 'name': 'Bill'}, {'personId': 'b36c4ad4-26d9-4f6d-860e-4d64df30174b', 'persistedFaceIds': ['002ce8d2-42df-4082-8dcb-78eebe12b667', '87b74572-1190-4ecd-88de-0c5a020409db', 'c39279d6-4fd7-4faf-b3af-3a5f3d20a198'], 'name': 'Angela'}, {'personId': 'ef263804-9976-456e-9554-2e1c39d2bde2', 'persistedFaceIds': ['1affc4b1-a428-4ba0-87ee-57f232596be9', '278ca2a5-aab9-46d9-a229-5c0d1da026fd', 'ec607745-2cb5-4ccd-b40b-b7d7d6d2ea94'], 'name': 'Elizabeth'}]


# Adicionando faces a pessoas

In [19]:
# Deleta, se for o caso, uma pessoa.
try:
    conn = http.client.HTTPSConnection('idfaceapi.cognitiveservices.azure.com')
    conn.request('DELETE', pessoa_endpoint+"/75066b3e-ea9a-4f26-93aa-836c8a2d865c", "", cabecalho) # personId
    response = conn.getresponse()
    jsonData = response.read()
    print(jsonData)
    conn.close()
except Exception as ex:
    print(ex)

b''


In [22]:
# Associa as pessoas as faces conforme diretório e lista(person_data).
import os

face_cabecalho = {
    'Ocp-Apim-Subscription-Key':subscription_key,
    'Content-Type':'application/octet-stream'
}

for pessoa in pessoas_dados: # Itera as pessoas.
    face_endpoint = '/face/v1.0/persongroups/{0}/persons/{1}/persistedFaces'.format(grupo_id, pessoa['personId'])
    for arquivo in os.listdir('People/'+pessoa['name']): # Itera os diretórios conforme os nomes das pessoas.
        with open('Pessoas/{0}/{1}'.format(pessoa['name'], arquivo), 'rb') as face_corpo:
            conn = http.client.HTTPSConnection('idfaceapi.cognitiveservices.azure.com')
            conn.request('POST', face_endpoint, face_corpo, face_cabecalho) # Associa a face a pessoa.
            response = conn.getresponse()
            jsonData = response.read()
            data = json.loads(jsonData)
            print(data)
            conn.close()

{'persistedFaceId': 'f456604b-a4be-4eb8-9680-6398a2aafb97'}
{'persistedFaceId': 'e80923d6-8865-4676-84c0-d953a0ad0fc3'}
{'persistedFaceId': '962828cd-c7aa-4cb3-b151-c17b7a04464d'}
{'persistedFaceId': '87b74572-1190-4ecd-88de-0c5a020409db'}
{'persistedFaceId': '002ce8d2-42df-4082-8dcb-78eebe12b667'}
{'persistedFaceId': 'c39279d6-4fd7-4faf-b3af-3a5f3d20a198'}
{'persistedFaceId': 'ec607745-2cb5-4ccd-b40b-b7d7d6d2ea94'}
{'persistedFaceId': '1affc4b1-a428-4ba0-87ee-57f232596be9'}
{'persistedFaceId': '278ca2a5-aab9-46d9-a229-5c0d1da026fd'}


In [24]:
# Lista novamente pessoas do grupo. Observe que temos agora persistedFaceIds.
try:
    conn = http.client.HTTPSConnection('idfaceapi.cognitiveservices.azure.com')
    conn.request('GET', pessoa_endpoint, "", cabecalho)
    response = conn.getresponse()
    jsonData = response.read()
    pessoas_dados = json.loads(jsonData)
    print(pessoas_dados)
    conn.close()
except Exception as ex:
    print(ex)

[{'personId': 'ab9243d6-97bb-40a2-9ad7-7f9166fc1c71', 'persistedFaceIds': ['962828cd-c7aa-4cb3-b151-c17b7a04464d', 'e80923d6-8865-4676-84c0-d953a0ad0fc3', 'f456604b-a4be-4eb8-9680-6398a2aafb97'], 'name': 'Bill'}, {'personId': 'b36c4ad4-26d9-4f6d-860e-4d64df30174b', 'persistedFaceIds': ['002ce8d2-42df-4082-8dcb-78eebe12b667', '87b74572-1190-4ecd-88de-0c5a020409db', 'c39279d6-4fd7-4faf-b3af-3a5f3d20a198'], 'name': 'Angela'}, {'personId': 'ef263804-9976-456e-9554-2e1c39d2bde2', 'persistedFaceIds': ['1affc4b1-a428-4ba0-87ee-57f232596be9', '278ca2a5-aab9-46d9-a229-5c0d1da026fd', 'ec607745-2cb5-4ccd-b40b-b7d7d6d2ea94'], 'name': 'Elizabeth'}]


# Identificando as pessoas

In [25]:
# Treina para o grupo de pessoas.
treino_cabecalho = {
    'Ocp-Apim-Subscription-Key':subscription_key
}
treino_endpoint = '/face/v1.0/persongroups/{0}/train'.format(grupo_id) 

try:
    conn = http.client.HTTPSConnection('idfaceapi.cognitiveservices.azure.com')
    conn.request('POST', treino_endpoint, '', treino_cabecalho)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

b''


In [28]:
# Identifica um indivíduo a partir de uma imagem.
identifica_endpoint = '/face/v1.0/identify'

identifica_cabecalho = {
    'Ocp-Apim-Subscription-Key':subscription_key
}

identifica_corpo = {
    'faceIds':['da0b71e5-f740-4d2b-b9e4-f4f544d1a7ac'], # Bill Gates
    'personGroupId':grupo_id
}

try:
    conn = http.client.HTTPSConnection('idfaceapi.cognitiveservices.azure.com')
    conn.request('POST', identifica_endpoint, str(identifica_corpo), identifica_cabecalho)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

[{'faceId': 'da0b71e5-f740-4d2b-b9e4-f4f544d1a7ac', 'candidates': [{'personId': 'ab9243d6-97bb-40a2-9ad7-7f9166fc1c71', 'confidence': 0.71294}]}]


In [32]:
identificada_pessoa = data[0]
encontrei = False
for candidato in identificada_pessoa['candidates']:
    for pessoa in pessoas_dados:
        if(pessoa['personId'] == candidato['personId']):
            print('Encontrei!')
            print('É ' + pessoa['name'])
            encontrei = True
            
if(encontrei == False):
    print('Esta imagem não pode ser identificada')

Encontrei!
É Bill
